# Practice with simple small graph

In [1]:
from dotenv import load_dotenv

_ = load_dotenv()

In [10]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.memory import MemorySaver

In [4]:
class AgentState(TypedDict):
    lnode: str
    pad: str
    count: Annotated[int, operator.add]

In [6]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode": "node_1",
            "count": 1}

def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode": "node_2",
            "count": 1}

def should_continue(state: AgentState) -> bool:
    return state["count"] < 3

The graph should go N1 -> N2 -> N1 but break

In [11]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)
builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, { True: "Node1", False: END})
builder.set_entry_point("Node1")

#memory = SqliteSaver.from_conn_string(":memory:")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [12]:
thread = {"configurable": {"thread_id": "1"}}

graph.invoke({"count": 0, "pad": "Hello"}, thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'pad': 'Hello', 'count': 4}

# Let's investigate

In [15]:
display(graph.get_state(thread))

StateSnapshot(values={'lnode': 'node_2', 'pad': 'Hello', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc1a-6a46-8004-4807462a87ee'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-08-12T11:25:01.914366+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-6fd4-8003-df0ed3cd81a3'}}, tasks=(), interrupts=())

In [16]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node_2', 'pad': 'Hello', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc1a-6a46-8004-4807462a87ee'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-08-12T11:25:01.914366+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-6fd4-8003-df0ed3cd81a3'}}, tasks=(), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'pad': 'Hello', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-6fd4-8003-df0ed3cd81a3'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-12T11:25:01.913692+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-64a8-8002-8d44223bf0d8'}}, tasks=(PregelTask(id='61e78077-0618-e2cc-88a0-66ebc59b14ab', name='Node2', path=('__pregel_pull',

In [17]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc1a-6a46-8004-4807462a87ee'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-6fd4-8003-df0ed3cd81a3'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-64a8-8002-8d44223bf0d8'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc16-6a04-8001-9c1eeef210fa'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc14-6e20-8000-ed6546805b09'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc12-651c-bfff-7eaf9c73fa9e'}} 0


In [18]:
graph.get_state(states[-3])

StateSnapshot(values={'lnode': 'node_1', 'pad': 'Hello', 'count': 1}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc16-6a04-8001-9c1eeef210fa'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-12T11:25:01.912723+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc14-6e20-8000-ed6546805b09'}}, tasks=(PregelTask(id='a803e457-082e-5763-33cb-0bf4e0ce992e', name='Node2', path=('__pregel_pull', 'Node2'), error=None, interrupts=(), state=None, result={'lnode': 'node_2', 'count': 1}),), interrupts=())

# Go back in time

In [19]:
graph.invoke(None, states[-3])

node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node_2', 'pad': 'Hello', 'count': 4}

In [20]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-783c-6270-8004-6ea844f1aa45'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-783a-607e-8003-ed139a52baf6'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-7838-6c10-8002-6ce795fcd473'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc1a-6a46-8004-4807462a87ee'}} 4
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-6fd4-8003-df0ed3cd81a3'}} 3
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc18-64a8-8002-8d44223bf0d8'}} 2
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc16-6a04-8001-9c1eeef210fa'}} 1
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0776ef-cc14-6e20-8000-ed6546805b09'}} 0
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkp

Notice how the new states are not in the history from the counts

In [21]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state,"\n")

StateSnapshot(values={'lnode': 'node_2', 'pad': 'Hello', 'count': 4}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-783c-6270-8004-6ea844f1aa45'}}, metadata={'source': 'loop', 'step': 4, 'parents': {}}, created_at='2025-08-12T11:36:31.052236+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-783a-607e-8003-ed139a52baf6'}}, tasks=(), interrupts=()) 

StateSnapshot(values={'lnode': 'node_1', 'pad': 'Hello', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-783a-607e-8003-ed139a52baf6'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-08-12T11:36:31.051373+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f077709-7838-6c10-8002-6ce795fcd473'}}, tasks=(PregelTask(id='568bc7ee-8f44-e88e-8006-5fd846d7ecef', name='Node2', path=('__pregel_pull',

# Modifying states